In [1]:
# Imports et al.
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytensor

from pymc_marketing.mmm import GeometricAdstock, LogisticSaturation
from pymc_marketing.mmm.additive_effect import LinearTrendEffect
from pymc_marketing.mmm.linear_trend import LinearTrend
from pymc_marketing.mmm.multidimensional import (
    MMM,
    MultiDimensionalBudgetOptimizerWrapper,
)
from pymc_marketing.paths import data_dir
from pymc_marketing.prior import Prior

pytensor.config.cxx = "/usr/bin/clang++"

warnings.filterwarnings("ignore", category=UserWarning)

az.style.use("arviz-darkgrid")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100
plt.rcParams["xtick.labelsize"] = 10
plt.rcParams["ytick.labelsize"] = 8

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

/Users/pablo.deroqueglovoapp.com/pymc-labs/pymc-marketing/pymc_marketing/mmm/multidimensional.py:68: FutureWarning: This functionality is experimental and subject to change. If you encounter any issues or have suggestions, please raise them at: https://github.com/pymc-labs/pymc-marketing/issues/new


In [2]:
seed: int = sum(map(ord, "mmm_multidimensional"))
rng: np.random.Generator = np.random.default_rng(seed=seed)

In [3]:
# Load the data
data_path = data_dir / "mmm_example.csv"
raw_data_df = pd.read_csv(data_path, parse_dates=["date_week"])
raw_data_df = raw_data_df.rename(columns={"date_week": "date"})

# Make data zero for one of the channel dimensions
# Create copies of the original data foo both geos
a_data_df = raw_data_df.copy().assign(geo="geo_a")
b_data_df = raw_data_df.copy().assign(geo="geo_b")

# Add noise to the target variable for the second geo
b_data_df["y"] = b_data_df["y"] + 500 * rng.normal(size=len(b_data_df))

# Concatenate the two datasets
data_df = pd.concat([a_data_df, b_data_df])

#! Make channel data zero for one of the dimensions
data_df.loc[data_df["geo"] == "geo_a", "x2"] = 0.0

In [4]:
channel_columns = ["x1", "x2"]
n_channels = len(channel_columns)

In [47]:
# Pooled Priors, nothing fancy
common_lam = Prior("LogNormal", mu=-1.5, sigma=0.5, dims=("channel"))
common_beta = Prior("LogNormal", mu=-1.5, sigma=0.5, dims=("channel"))
saturation_priors = {
    "beta": common_beta,
    "lam": common_lam,
}
saturation = LogisticSaturation(priors=saturation_priors)

alpha = Prior("Beta", alpha=1, beta=5, dims=("channel"))
adstock = GeometricAdstock(l_max=8, priors={"alpha": alpha})

model_config = {
    "intercept": Prior(
        "TruncatedNormal", mu=2, sigma=0.5, lower=0.35, upper=2.5, dims="geo"
    ),
}

In [48]:
# Base MMM model specification
pooled_mmm = MMM(
    date_column="date",
    target_column="y",
    channel_columns=["x1", "x2"],
    control_columns=["event_1", "event_2"],
    dims=("geo",),
    scaling={
        "channel": {"method": "max", "dims": ()},
        "target": {"method": "max", "dims": ()},
    },
    adstock=adstock,
    saturation=saturation,
    yearly_seasonality=2,
    model_config=model_config,
)

# Adding LinearTrend as in the example notebook
linear_trend = LinearTrend(
    priors={
        "delta": Prior(
            "Laplace",
            mu=0,
            b=Prior("HalfNormal", sigma=0.2),
            dims=("changepoint", "geo"),
        ),
    },
    n_changepoints=5,
    include_intercept=False,
    dims=("geo"),
)

linear_trend_effect = LinearTrendEffect(linear_trend, prefix="trend")

pooled_mmm.mu_effects.append(linear_trend_effect)

In [49]:
# X,y
x_train = data_df.drop(columns=["y"])

y_train = data_df["y"]

# Building the model
pooled_mmm.build_model(X=x_train, y=y_train)

# Add deterministics
pooled_mmm.add_original_scale_contribution_variable(
    var=[
        "channel_contribution",
        "control_contribution",
        "intercept_contribution",
        "yearly_seasonality_contribution",
    ]
)

In [50]:
scalers = pooled_mmm.get_scales_as_xarray()

In [51]:
# Skipping prior checks for brevity, but nothing odd
# prior_predictive = mmm.sample_prior_predictive(X=x_train, y=y_train, samples=1_000)


# Fitting the model and predicting
pooled_mmm.fit(
    X=x_train,
    y=y_train,
    chains=4,
    target_accept=0.8,
    nuts_sampler="nutpie",
    random_seed=rng,
)

Progress,Draws,Divergences,Step Size,Gradients/Draw
,2000,0,0.17,31
,2000,0,0.16,31
,2000,0,0.16,31
,2000,0,0.15,31


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data
	> fit_data

Warmup iterations saved (warmup_*).

In [52]:
pooled_mmm.sample_posterior_predictive(
    X=x_train,
    extend_idata=True,
    combined=True,
    random_seed=rng,
)

Sampling: [y]


Output()

<xarray.Dataset> Size: 12MB
Dimensions:  (date: 179, geo: 2, sample: 4000)
Coordinates:
  * date     (date) datetime64[ns] 1kB 2018-04-02 2018-04-09 ... 2021-08-30
  * geo      (geo) <U5 40B 'geo_a' 'geo_b'
  * sample   (sample) object 32kB MultiIndex
  * chain    (sample) int64 32kB 0 0 0 0 0 0 0 0 0 0 0 ... 3 3 3 3 3 3 3 3 3 3 3
  * draw     (sample) int64 32kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
Data variables:
    y        (date, geo, sample) float64 11MB 0.5109 0.5118 ... 0.6325 0.5456
Attributes:
    created_at:                 2025-06-17T18:09:57.414223+00:00
    arviz_version:              0.21.0
    inference_library:          pymc
    inference_library_version:  5.23.0

In [56]:
# No divergences
pooled_mmm.idata.sample_stats.diverging.sum().item()  # this evaluates to 0

In [57]:
az.summary(
    pooled_mmm.idata, var_names=["saturation_lam", "saturation_beta", "adstock_alpha"]
)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
saturation_lam[x1],2.278,0.725,0.906,3.502,0.035,0.014,445.0,706.0,1.01
saturation_lam[x2],0.688,0.256,0.274,1.149,0.007,0.007,1426.0,1822.0,1.00
saturation_beta[x1],0.532,0.178,0.344,0.857,0.009,0.013,533.0,647.0,1.01
saturation_beta[x2],0.696,0.245,0.300,1.139,0.006,0.006,1434.0,1838.0,1.00
adstock_alpha[x1],0.367,0.047,0.281,0.456,0.001,0.001,2529.0,2952.0,1.00
adstock_alpha[x2],0.094,0.062,0.000,0.198,0.001,0.001,3079.0,1604.0,1.00


We have a pooled model, one coefficient per channel, and the same saturation and adstock parameters for both geos.

In [58]:
# You can follow along the notebook, nothing is odd.
# But then:
optimizable_model = MultiDimensionalBudgetOptimizerWrapper(
    model=pooled_mmm, start_date="2021-10-01", end_date="2021-12-31"
)

In [59]:
allocation_xarray, scipy_opt_result = optimizable_model.optimize_budget(
    budget=10,  # Total budget to allocate here is spend in Millions
)

MinimizeException: Optimization failed: Inequality constraints incompatible

## Hierarchical model

In [60]:
## Hierarchical Priors
hierarchical_lam = Prior(
    "Gamma",
    mu=Prior("LogNormal", mu=np.log(3), sigma=np.log(1.5), dims="channel"),
    sigma=Prior("LogNormal", mu=np.log(1), sigma=np.log(1.5), dims="channel"),
    dims=("channel", "geo"),
)
hierarchical_beta = Prior(
    "Gamma",
    mu=Prior("LogNormal", mu=np.log(3), sigma=np.log(1.5), dims="channel"),
    sigma=Prior("LogNormal", mu=np.log(1), sigma=np.log(1.5), dims="channel"),
    dims=("channel", "geo"),
)

saturation_priors = {
    "lam": hierarchical_lam,
    "beta": hierarchical_beta,
}
saturation = LogisticSaturation(priors=saturation_priors)

alpha = Prior("Beta", alpha=1, beta=5, dims=("channel"))
adstock = GeometricAdstock(l_max=8, priors={"alpha": alpha})

model_config = {
    "intercept": Prior(
        "TruncatedNormal", mu=2, sigma=0.5, lower=0.35, upper=2.5, dims="geo"
    ),
}

In [61]:
saturation

LogisticSaturation(prefix='saturation', priors={'lam': Prior("Gamma", mu=Prior("LogNormal", mu=1.0986122886681098, sigma=0.4054651081081644, dims="channel"), sigma=Prior("LogNormal", mu=0.0, sigma=0.4054651081081644, dims="channel"), dims=("channel", "geo")), 'beta': Prior("Gamma", mu=Prior("LogNormal", mu=1.0986122886681098, sigma=0.4054651081081644, dims="channel"), sigma=Prior("LogNormal", mu=0.0, sigma=0.4054651081081644, dims="channel"), dims=("channel", "geo"))})

In [62]:
# Base MMM model specification
hier_mmm = MMM(
    date_column="date",
    target_column="y",
    channel_columns=["x1", "x2"],
    control_columns=["event_1", "event_2"],
    dims=("geo",),
    scaling={
        "channel": {"method": "max", "dims": ()},
        "target": {"method": "max", "dims": ()},
    },
    adstock=adstock,
    saturation=saturation,
    yearly_seasonality=2,
    model_config=model_config,
)

# Adding LinearTrend as in the example notebook
linear_trend = LinearTrend(
    priors={
        "delta": Prior(
            "Laplace",
            mu=0,
            b=Prior("HalfNormal", sigma=0.2),
            dims=("changepoint", "geo"),
        ),
    },
    n_changepoints=5,
    include_intercept=False,
    dims=("geo"),
)

linear_trend_effect = LinearTrendEffect(linear_trend, prefix="trend")

hier_mmm.mu_effects.append(linear_trend_effect)

In [63]:
# X,y
x_train = data_df.drop(columns=["y"])

y_train = data_df["y"]

# Building the model
hier_mmm.build_model(X=x_train, y=y_train)

# Add deterministics
hier_mmm.add_original_scale_contribution_variable(
    var=[
        "channel_contribution",
        "control_contribution",
        "intercept_contribution",
        "yearly_seasonality_contribution",
    ]
)

In [64]:
scalers = hier_mmm.get_scales_as_xarray()

In [65]:
# Skipping prior checks for brevity, but nothing odd
# prior_predictive = mmm.sample_prior_predictive(X=x_train, y=y_train, samples=1_000)


# Fitting the model and predicting
hier_mmm.fit(
    X=x_train,
    y=y_train,
    chains=4,
    target_accept=0.8,
    nuts_sampler="nutpie",
    random_seed=rng,
)

Progress,Draws,Divergences,Step Size,Gradients/Draw
,2000,0,0.15,63
,2000,0,0.15,63
,2000,0,0.15,127
,2000,1,0.17,63


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data
	> fit_data

Warmup iterations saved (warmup_*).

In [66]:
hier_mmm.sample_posterior_predictive(
    X=x_train,
    extend_idata=True,
    combined=True,
    random_seed=rng,
)

Sampling: [y]


Output()

<xarray.Dataset> Size: 12MB
Dimensions:  (date: 179, geo: 2, sample: 4000)
Coordinates:
  * date     (date) datetime64[ns] 1kB 2018-04-02 2018-04-09 ... 2021-08-30
  * geo      (geo) <U5 40B 'geo_a' 'geo_b'
  * sample   (sample) object 32kB MultiIndex
  * chain    (sample) int64 32kB 0 0 0 0 0 0 0 0 0 0 0 ... 3 3 3 3 3 3 3 3 3 3 3
  * draw     (sample) int64 32kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
Data variables:
    y        (date, geo, sample) float64 11MB 0.4723 0.5232 ... 0.6805 0.6139
Attributes:
    created_at:                 2025-06-17T18:11:32.816789+00:00
    arviz_version:              0.21.0
    inference_library:          pymc
    inference_library_version:  5.23.0

In [67]:
az.summary(
    hier_mmm.idata, var_names=["saturation_lam", "saturation_beta", "adstock_alpha"]
)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"saturation_lam[x1, geo_a]",3.153,0.725,1.793,4.504,0.028,0.025,712.0,535.0,1.01
"saturation_lam[x1, geo_b]",3.493,0.657,2.295,4.709,0.021,0.023,1055.0,911.0,1.00
"saturation_lam[x2, geo_a]",2.786,1.888,0.001,5.844,0.065,0.055,541.0,396.0,1.01
"saturation_lam[x2, geo_b]",2.376,1.828,0.047,5.401,0.095,0.032,290.0,643.0,1.01
"saturation_beta[x1, geo_a]",0.444,0.102,0.318,0.580,0.005,0.016,912.0,515.0,1.01
"saturation_beta[x1, geo_b]",0.382,0.065,0.300,0.475,0.003,0.013,906.0,877.0,1.00
"saturation_beta[x2, geo_a]",1.986,1.781,0.000,4.821,0.039,0.066,1123.0,1029.0,1.00
"saturation_beta[x2, geo_b]",0.771,1.148,0.150,3.113,0.074,0.074,295.0,687.0,1.01
adstock_alpha[x1],0.351,0.044,0.270,0.431,0.001,0.001,2286.0,2901.0,1.00
adstock_alpha[x2],0.121,0.066,0.001,0.233,0.001,0.001,2696.0,1771.0,1.00


Hierarchical model, one param per channel per geo

In [68]:
# No divergences
hier_mmm.idata.sample_stats.diverging.sum().item()  # this evaluates to 0

In [69]:
# You can follow along the notebook, nothing is odd.
# But then:
optimizable_model = MultiDimensionalBudgetOptimizerWrapper(
    model=hier_mmm, start_date="2021-10-01", end_date="2021-12-31"
)

In [70]:
allocation_xarray, scipy_opt_result = optimizable_model.optimize_budget(
    budget=10,  # Total budget to allocate here is spend in Millions
)

MinimizeException: Optimization failed: Inequality constraints incompatible